In [1]:
import sys
sys.path.append("..")
import numpy as np
import pandas as pd
from project.ship import Ship
from project.tests import Tests
from project.botma import TMA
import time
from pathlib import Path
from datetime import datetime

In [2]:
# Класс для сохранения результатов
tests = Tests('test1')
Path("../tests/test1/d20").mkdir(parents=True, exist_ok=True)
print("Время последнего моделирования:", datetime.now().strftime("%d/%m/%Y %H:%M"))

# Создаем наблюдателя
observer_x, observer_y, observer_course, observer_velocity = 0, 0, 0, 5
observer = Ship('Наблюдатель', observer_x, observer_y, observer_course,
                observer_velocity, verbose=True)

Время последнего моделирования: 26/03/2021 04:29


In [3]:
# Моделирование траектории наблюдателя
observer.forward_movement(3 * 60)
observer.change_course(270, 'left', omega=0.5)
observer.forward_movement(5 * 60)
observer.change_course(90, 'right', omega=0.5)
observer.forward_movement(3 * 60)

Наблюдатель движется прямо по курсу 0.0° 180с
Наблюдатель перешёл на курс 270.0° за 180с
Наблюдатель движется прямо по курсу 270.0° 300с
Наблюдатель перешёл на курс 90.0° за 360с
Наблюдатель движется прямо по курсу 90.0° 180с


In [4]:
def target_func(seed=None):
    np.random.seed(seed)
    b = 0
    d = 20
    c = np.random.uniform(0, 180)
    v = np.random.uniform(3, 15)
    return [b, d, c, v]
results = []
p0 = [0., 25., 90., 7.]
std_arr = [0., 0.1, 0.2, 0.3, 0.5, 1.]
precisions_1 = ['{0:.1f}', '{0:.3f}', '{0:.3f}', '{0:.2f}', '{0:.1f}', '{0:.3f}', '{0:.3f}', '{0:.2f}', '{0:.1f}', '{0:.3f}', '{0:.3f}', '{0:.2f}',
'{0:.2f}','{0:.3f}', '{0:.2f}','{0:.3f}','{0:.2f}', '{0:.2f}', '{0:.2f}', '{0:.2f}', '{0:.2f}', '{0:.2f}', '{:.0f}', '{0:.4f}', '{0:.0f}', '{0:.0f}']
precisions_2 = ['{0:.1f}', '{0:.3f}', '{0:.1f}', '{0:.2f}', '{0:.1f}', '{0:.3f}', '{0:.1f}', '{0:.2f}', '{0:.2f}','{0:.3f}', '{0:.2f}','{0:.3f}', '{0:.2f}', '{0:.2f}', '{0:.0f}', '{0:.4f}']
precisions_3 = ['{0:.0f}', '{0:.0f}', '{0:.0f}', '{0:.0f}', '{0:.0f}', '{0:.0f}', '{0:.0f}', '{0:.0f}', '{0:.0f}', '{0:.2f}', '{0:.0f}', '{0:.0f}', '{0:.0f}', '{0:.0f}', '{0:.4f}', '{0:.4f}', '{0:.1f}']

In [5]:
for std in std_arr:

    tma = TMA(observer, std=np.radians(std), tau=2, verbose=True)
    algorithm_name = 'ММП v2'
    res = [algorithm_name]
    dict_results = tma.swarm(algorithm_name = algorithm_name, n=100, target_func=target_func, p0=p0, seeded=True, verbose=True)
    df = tests.get_df(dict_results)

    print('Показатели Монте-Карло:')
    for i in range(4):
        success = df['Успех'].apply(lambda x: x[i])
        success_rate = 100 * success.mean()
        success_min = min(success)
        print('Вариант {}: Рэф = {:.0f}%, Рэф_мин = {:.0f}'.format(i, success_rate, success_min))
        res.extend([success_rate, success_min])
    k_a_mean = df['Ка'].mean()
    nf_mean = df['Nf'].mean()
    nf_max = max(df['Nf'])
    iter_mean = df['Iter'].mean()
    iter_max = max(df['Iter'])
    t_mean = df['t'].mean()
    t_max = max(df['t'])
    std = np.degrees(tma.standart_deviation)
    print('Ка = {:.2f}'.format(k_a_mean))
    print('Nf = {:.0f}, Nf_max = {:.0f}'.format(nf_mean, nf_max))
    print('Iter = {:.0f}, Iter_max = {:.0f}'.format(iter_mean, iter_max))
    print('t = {:.4f}, t_max = {:.4f}'.format(t_mean, t_max))

    d_df = df.copy()
    for i, j in enumerate(df.columns):
        try: d_df[j] = df[j].apply(lambda x: precisions_1[i].format(np.float64(x)))
        except(TypeError): pass
    display(d_df.drop(['СКО X', 'СКО Y', 'СКО VX', 'СКО VY', 'Кб', 'П0_апр', 'Д0_апр', 'К0_апр', 'V0_апр'], axis=1).head())

    res.extend([k_a_mean, nf_mean, nf_max, iter_mean, iter_max, t_mean, t_max])
    results.append(res)
    tests.save_df(df, name='../tests/test1/d20/ЛМ, ' + str(std) + '.xlsx')
    print('_'*40)

df_lm = pd.DataFrame(results, columns=['Алгоритм', 'В11', 'В12', 'В21', 'В22', 'В31', 'В32', 'В41', 'В42', 'Ка', 'Nf', 'Nf_max', 'Iter', 'Iter_max', 't', 't_max'])
df_lm['СКО'] = std_arr
results = []

СКОп = 0.0, tau = 2, end_time = 1200
Алгоритм:: ММП v2
Моделирование 100 результатов закончено за t = 0.6 с
Показатели Монте-Карло:
Вариант 0: Рэф = 100%, Рэф_мин = 1
Вариант 1: Рэф = 100%, Рэф_мин = 1
Вариант 2: Рэф = 100%, Рэф_мин = 1
Вариант 3: Рэф = 100%, Рэф_мин = 1
Ка = 0.00
Nf = 10, Nf_max = 24
Iter = 8, Iter_max = 19
t = 0.0021, t_max = 0.0076


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Успех,t,Nf,Iter
0,0.0,20.000,98.786,11.58,0.0,20.000,98.786,11.58,44.33,21.333,44.33,21.332,0.00,"[1, 1, 1, 1]",0.0034,10,9
1,0.0,20.000,75.064,11.64,0.0,20.000,75.064,11.64,34.96,25.605,34.96,25.605,0.00,"[1, 1, 1, 1]",0.0023,10,9
2,0.0,20.000,78.479,3.31,360.0,20.000,78.479,3.31,15.57,18.869,15.57,18.869,0.00,"[1, 1, 1, 1]",0.0017,8,7
3,0.0,20.000,99.144,11.50,0.0,20.000,99.144,11.50,44.24,21.203,44.24,21.203,0.00,"[1, 1, 1, 1]",0.0025,10,9
4,0.0,20.000,174.065,9.57,360.0,20.000,174.065,9.57,21.56,6.414,21.56,6.414,0.00,"[1, 1, 1, 1]",0.0019,9,8


_________________________________________________
СКОп = 0.1, tau = 2, end_time = 1200
Алгоритм:: ММП v2
Моделирование 100 результатов закончено за t = 0.5 с
Показатели Монте-Карло:
Вариант 0: Рэф = 100%, Рэф_мин = 1
Вариант 1: Рэф = 100%, Рэф_мин = 1
Вариант 2: Рэф = 100%, Рэф_мин = 1
Вариант 3: Рэф = 100%, Рэф_мин = 1
Ка = 0.01
Nf = 10, Nf_max = 24
Iter = 8, Iter_max = 19
t = 0.0020, t_max = 0.0049


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Успех,t,Nf,Iter
0,0.0,20.000,98.786,11.58,360.0,20.062,98.701,11.65,44.33,21.333,44.35,21.439,0.01,"[1, 1, 1, 1]",0.0027,10,9
1,0.0,20.000,75.064,11.64,0.0,19.908,75.064,11.56,34.96,25.605,34.94,25.456,0.01,"[1, 1, 1, 1]",0.0020,10,9
2,0.0,20.000,78.479,3.31,0.0,20.046,79.766,3.29,15.57,18.869,15.58,18.823,0.01,"[1, 1, 1, 1]",0.0016,8,7
3,0.0,20.000,99.144,11.50,0.0,19.906,99.296,11.40,44.24,21.203,44.23,21.045,0.01,"[1, 1, 1, 1]",0.0027,10,9
4,0.0,20.000,174.065,9.57,0.0,19.995,174.132,9.58,21.56,6.414,21.55,6.388,0.01,"[1, 1, 1, 1]",0.0018,9,8


_________________________________________________
СКОп = 0.2, tau = 2, end_time = 1200
Алгоритм:: ММП v2
Моделирование 100 результатов закончено за t = 0.5 с
Показатели Монте-Карло:
Вариант 0: Рэф = 98%, Рэф_мин = 0
Вариант 1: Рэф = 100%, Рэф_мин = 1
Вариант 2: Рэф = 100%, Рэф_мин = 1
Вариант 3: Рэф = 100%, Рэф_мин = 1
Ка = 0.04
Nf = 10, Nf_max = 24
Iter = 8, Iter_max = 19
t = 0.0021, t_max = 0.0048


...

_________________________________________________
СКОп = 0.5, tau = 2, end_time = 1200
Алгоритм:: ММП v2
Моделирование 100 результатов закончено за t = 0.6 с
Показатели Монте-Карло:
Вариант 0: Рэф = 79%, Рэф_мин = 0
Вариант 1: Рэф = 97%, Рэф_мин = 0
Вариант 2: Рэф = 97%, Рэф_мин = 0
Вариант 3: Рэф = 98%, Рэф_мин = 0
Ка = 0.25
Nf = 10, Nf_max = 24
Iter = 8, Iter_max = 19
t = 0.0022, t_max = 0.0126


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Успех,t,Nf,Iter
0,0.0,20.000,98.786,11.58,359.9,20.317,98.369,11.95,44.33,21.333,44.45,21.871,0.25,"[1, 1, 1, 1]",0.0028,10,9
1,0.0,20.000,75.064,11.64,0.0,19.551,75.064,11.25,34.96,25.605,34.86,24.879,0.25,"[1, 1, 1, 1]",0.0020,10,9
2,0.0,20.000,78.479,3.31,0.1,20.231,85.105,3.21,15.57,18.869,15.65,18.634,0.26,"[0, 1, 1, 1]",0.0016,8,7
3,0.0,20.000,99.144,11.50,0.1,19.541,99.916,11.02,44.24,21.203,44.19,20.426,0.26,"[1, 1, 1, 1]",0.0028,10,9
4,0.0,20.000,174.065,9.57,0.0,19.976,174.394,9.64,21.56,6.414,21.48,6.284,0.24,"[1, 1, 1, 1]",0.0018,9,8


_________________________________________________
СКОп = 1.0, tau = 2, end_time = 1200
Алгоритм:: ММП v2
Моделирование 100 результатов закончено за t = 0.5 с
Показатели Монте-Карло:
Вариант 0: Рэф = 44%, Рэф_мин = 0
Вариант 1: Рэф = 80%, Рэф_мин = 0
Вариант 2: Рэф = 81%, Рэф_мин = 0
Вариант 3: Рэф = 92%, Рэф_мин = 0
Ка = 0.99
Nf = 10, Nf_max = 24
Iter = 8, Iter_max = 19
t = 0.0020, t_max = 0.0051


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Успех,t,Nf,Iter
0,0.0,20.000,98.786,11.58,359.8,20.645,97.971,12.33,44.33,21.333,44.57,22.426,1.00,"[0, 1, 1, 1]",0.0024,10,9
1,0.0,20.000,75.064,11.64,0.0,19.126,75.065,10.88,34.96,25.605,34.76,24.192,0.99,"[0, 1, 1, 1]",0.0018,9,8
2,0.0,20.000,78.479,3.31,0.1,20.460,92.130,3.14,15.57,18.869,15.73,18.392,1.05,"[0, 0, 0, 0]",0.0017,8,7
3,0.0,20.000,99.144,11.50,0.2,19.106,100.717,10.57,44.24,21.203,44.13,19.689,1.04,"[0, 1, 1, 1]",0.0019,10,9
4,0.0,20.000,174.065,9.57,0.0,19.955,174.709,9.71,21.56,6.414,21.40,6.156,0.94,"[1, 1, 1, 1]",0.0018,9,8


_________________________________________________


In [6]:
for std in std_arr:

    tma = TMA(observer, std=np.radians(std), tau=2, verbose=True)
    algorithm_name = 'Метод N пеленгов'
    res = [algorithm_name]
    dict_results = tma.swarm(n=100, algorithm_name=algorithm_name, target_func=target_func, p0=p0, seeded=True, verbose=True)
    df = tests.get_df(dict_results).dropna(axis=1)

    print('Показатели Монте-Карло:')
    for i in range(4):
        success = df['Успех'].apply(lambda x: x[i])
        success_rate = 100 * success.mean()
        success_min = min(success)
        print('Вариант {}: Рэф = {:.0f}%, Рэф_мин = {:.0f}'.format(i, success_rate, success_min))
        res.extend([success_rate, success_min])
    k_a_mean = df['Ка'].mean()
    t_mean = df['t'].mean()
    t_max = max(df['t'])
    std = np.degrees(tma.standart_deviation)
    print('Ка = {:.2f}'.format(k_a_mean))
    print('t = {:.4f}, t_max = {:.4f}'.format(t_mean, t_max))
    
    d_df = df.copy()
    for i, j in enumerate(df.columns):
        try: d_df[j] = df[j].apply(lambda x: precisions_1[i].format(np.float64(x)))
        except(TypeError): pass
    display(d_df.drop(['Кб'], axis=1).head())

    res.extend([k_a_mean, t_mean, t_max])
    results.append(res)
    tests.save_df(df, name='../tests/test1/d20/N, ' + str(np.degrees(tma.standart_deviation)) + '.xlsx')
    print('_'*40)

df_n = pd.DataFrame(results, columns=['Алгоритм', 'В11', 'В12', 'В21', 'В22', 'В31', 'В32', 'В41', 'В42', 'Ка', 't', 't_max'])
df_n['СКО'] = std_arr

СКОп = 0.0, tau = 2, end_time = 1200
Алгоритм:: Метод N пеленгов
Моделирование 100 результатов закончено за t = 0.4 с
Показатели Монте-Карло:
Вариант 0: Рэф = 100%, Рэф_мин = 1
Вариант 1: Рэф = 100%, Рэф_мин = 1
Вариант 2: Рэф = 100%, Рэф_мин = 1
Вариант 3: Рэф = 100%, Рэф_мин = 1
Ка = 0.00
t = 0.0004, t_max = 0.0077


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Успех,t
0,0.0,20.000,98.786,11.58,0.0,20.000,98.786,11.58,44.3,21.333,44.327,21.33,0.00,"[1, 1, 1, 1]",0.000
1,0.0,20.000,75.064,11.64,0.0,20.000,75.064,11.64,35.0,25.605,34.958,25.60,0.00,"[1, 1, 1, 1]",0.000
2,0.0,20.000,78.479,3.31,0.0,20.000,78.479,3.31,15.6,18.869,15.567,18.87,0.00,"[1, 1, 1, 1]",0.000
3,0.0,20.000,99.144,11.50,0.0,20.000,99.144,11.50,44.2,21.203,44.238,21.20,0.00,"[1, 1, 1, 1]",0.000
4,0.0,20.000,174.065,9.57,0.0,20.000,174.065,9.57,21.6,6.414,21.564,6.41,0.00,"[1, 1, 1, 1]",0.000


_________________________________________________
СКОп = 0.1, tau = 2, end_time = 1200
Алгоритм:: Метод N пеленгов
Моделирование 100 результатов закончено за t = 0.3 с
Показатели Монте-Карло:
Вариант 0: Рэф = 76%, Рэф_мин = 0
Вариант 1: Рэф = 95%, Рэф_мин = 0
Вариант 2: Рэф = 95%, Рэф_мин = 0
Вариант 3: Рэф = 99%, Рэф_мин = 0
Ка = 0.01
t = 0.0002, t_max = 0.0006


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Успех,t
0,0.0,20.000,98.786,11.58,359.8,20.298,97.853,11.99,44.3,21.333,44.358,21.98,0.01,"[1, 1, 1, 1]",0.000
1,0.0,20.000,75.064,11.64,359.8,20.017,74.176,11.93,35.0,25.605,34.942,25.99,0.01,"[1, 1, 1, 1]",0.000
2,0.0,20.000,78.479,3.31,0.0,19.971,81.630,3.20,15.6,18.869,15.557,18.60,0.01,"[1, 1, 1, 1]",0.000
3,0.0,20.000,99.144,11.50,359.8,20.229,98.149,11.85,44.2,21.203,44.240,21.77,0.01,"[1, 1, 1, 1]",0.000
4,0.0,20.000,174.065,9.57,360.0,19.960,174.131,9.56,21.6,6.414,21.530,6.38,0.01,"[1, 1, 1, 1]",0.000


_________________________________________________
СКОп = 0.2, tau = 2, end_time = 1200
Алгоритм:: Метод N пеленгов
Моделирование 100 результатов закончено за t = 0.3 с
Показатели Монте-Карло:
Вариант 0: Рэф = 57%, Рэф_мин = 0
Вариант 1: Рэф = 74%, Рэф_мин = 0
Вариант 2: Рэф = 75%, Рэф_мин = 0
Вариант 3: Рэф = 88%, Рэф_мин = 0
Ка = 0.04
t = 0.0002, t_max = 0.0006


...

_________________________________________________
СКОп = 0.5, tau = 2, end_time = 1200
Алгоритм:: Метод N пеленгов
Моделирование 100 результатов закончено за t = 0.3 с
Показатели Монте-Карло:
Вариант 0: Рэф = 0%, Рэф_мин = 0
Вариант 1: Рэф = 11%, Рэф_мин = 0
Вариант 2: Рэф = 18%, Рэф_мин = 0
Вариант 3: Рэф = 36%, Рэф_мин = 0
Ка = 0.32
t = 0.0002, t_max = 0.0006


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Успех,t
0,0.0,20.000,98.786,11.58,359.1,17.690,93.987,10.67,44.3,21.333,43.951,19.70,0.38,"[0, 1, 1, 1]",0.000
1,0.0,20.000,75.064,11.64,359.2,16.778,69.926,10.36,35.0,25.605,34.389,22.33,0.39,"[0, 0, 0, 1]",0.000
2,0.0,20.000,78.479,3.31,0.2,18.446,102.336,2.45,15.6,18.869,15.139,15.75,0.30,"[0, 0, 0, 0]",0.000
3,0.0,20.000,99.144,11.50,359.1,17.076,93.936,10.10,44.2,21.203,43.642,18.83,0.46,"[0, 0, 0, 1]",0.000
4,0.0,20.000,174.065,9.57,360.0,19.028,175.258,9.18,21.6,6.414,20.907,5.81,0.26,"[0, 1, 1, 1]",0.000


_________________________________________________
СКОп = 1.0, tau = 2, end_time = 1200
Алгоритм:: Метод N пеленгов
Моделирование 100 результатов закончено за t = 0.3 с
Показатели Монте-Карло:
Вариант 0: Рэф = 0%, Рэф_мин = 0
Вариант 1: Рэф = 0%, Рэф_мин = 0
Вариант 2: Рэф = 0%, Рэф_мин = 0
Вариант 3: Рэф = 0%, Рэф_мин = 0
Ка = 2.00
t = 0.0002, t_max = 0.0004


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Успех,t
0,0.0,20.000,98.786,11.58,358.2,10.399,86.330,5.38,44.3,21.333,41.679,10.97,3.19,"[0, 0, 0, 0]",0.000
1,0.0,20.000,75.064,11.64,358.4,10.385,61.005,5.94,35.0,25.605,32.357,13.28,2.77,"[0, 0, 0, 0]",0.000
2,0.0,20.000,78.479,3.31,0.4,15.041,149.526,2.01,15.6,18.869,13.581,10.65,1.62,"[0, 0, 0, 0]",0.000
3,0.0,20.000,99.144,11.50,358.2,9.793,85.167,4.84,44.2,21.203,40.964,10.14,3.81,"[0, 0, 0, 0]",0.000
4,0.0,20.000,174.065,9.57,359.9,16.678,178.392,8.21,21.6,6.414,18.762,4.45,1.43,"[0, 0, 0, 0]",0.000


_________________________________________________


In [7]:
df = pd.concat([df_lm, df_n])
df['Ка'] = df['Ка'].apply(lambda x: np.sqrt(np.float64(x)))
df = df.rename(columns={'Ка' : 'sqrt(Ка)'})
for i, j in enumerate(df.columns):
        try: df[j] = df[j].apply(lambda x: precisions_3[i].format(np.float64(x)))
        except(ValueError): pass

print('Сводная таблица:')
df = df.groupby('СКО').apply(lambda a: a.drop('СКО', axis=1)[:]).reset_index().drop('level_1', axis=1)
df

Сводная таблица:


,СКО,Алгоритм,В11,В12,В21,В22,В31,В32,В41,В42,sqrt(Ка),Nf,Nf_max,Iter,Iter_max,t,t_max
0,0.0,ММП v2,100,1,100,1,100,1,100,1,0.00,10,24,8,19,0.0021,0.0076
1,0.0,Метод N пеленгов,100,1,100,1,100,1,100,1,0.00,nan,nan,nan,nan,0.0004,0.0077
2,0.1,ММП v2,100,1,100,1,100,1,100,1,0.10,10,24,8,19,0.0020,0.0049
3,0.1,Метод N пеленгов,76,0,95,0,95,0,99,0,0.10,nan,nan,nan,nan,0.0002,0.0006
4,0.2,ММП v2,98,0,100,1,100,1,100,1,0.20,10,24,8,19,0.0021,0.0048
5,0.2,Метод N пеленгов,57,0,74,0,75,0,88,0,0.21,nan,nan,nan,nan,0.0002,0.0006
6,0.3,ММП v2,96,0,98,0,98,0,100,1,0.30,10,24,8,19,0.0021,0.0056
7,0.3,Метод N пеленгов,32,0,58,0,58,0,69,0,0.32,nan,nan,nan,nan,0.0002,0.0004
8,0.5,ММП v2,79,0,97,0,97,0,98,0,0.50,10,24,8,19,0.0022,0.0126
9,0.5,Метод N пеленгов,0,0,11,0,18,0,36,0,0.57,nan,nan,nan,nan,0.0002,0.0006


In [8]:
import docx
doc = docx.Document()
t = doc.add_table(df.shape[0]+1, df.shape[1])
for j in range(df.shape[-1]):
    t.cell(0,j).text = df.columns[j]
for i in range(df.shape[0]):
    for j in range(df.shape[-1]):
        t.cell(i+1,j).text = str(df.values[i,j])
doc.save('../tests/test1/d20/Сводная таблица.docx')